**IMPORT LIBRARIES**

In [23]:
from queue import PriorityQueue
import datetime
import tracemalloc


**GLOBAL VARIABLES**

In [24]:
goalstate2 = {1:0,4:1,7:2,2:3,5:4,8:5,3:6,6:7,0:8}

**PUZZLE CLASS**

In [25]:
class Puzzle:
    def __init__(self, state, position_0, parent_cost, path_cost=0, move=None, parent=None):
        self.state = state
        self.path_cost = path_cost + parent_cost
        self.move = move
        self.position_0 = position_0
        self.parent = parent

    def __lt__(self, other):
        return self.path_cost < other.path_cost

    def move_up(self, n, parent_cost, path_cost, parent, child_expand):
        i = parent.position_0 // n
        j = parent.position_0 % n
        if i <= 0:
            return None
        up_state = parent.state.copy()
        up_state[(i - 1) * n + j], up_state[i * n + j] = up_state[i * n + j], up_state[(i - 1) * n + j]
        temp = Puzzle(up_state, (i - 1) * n + j, parent_cost, path_cost, "up", parent)
        child_expand.append(temp)

    def move_down(self, n, parent_cost, path_cost, parent, child_expand):
        i = parent.position_0 // n
        j = parent.position_0 % n
        if i >= n - 1:
            return None
        down_state = parent.state.copy()
        down_state[(i + 1) * n + j], down_state[i * n + j] = down_state[i * n + j], down_state[(i + 1) * n + j]
        temp = Puzzle(down_state, (i + 1) * n + j, parent_cost, path_cost, "down", parent)
        child_expand.append(temp)

    def move_left(self, n, parent_cost, path_cost, parent, child_expand):
        i = parent.position_0 // n
        j = parent.position_0 % n
        if j <= 0:
            return None
        left_state = parent.state.copy()
        left_state[i * n + j], left_state[i * n + j - 1] = left_state[i * n + j - 1], left_state[i * n + j]
        temp = Puzzle(left_state, i * n + j - 1, parent_cost, path_cost, "left", parent)
        child_expand.append(temp)

    def move_right(self, n, parent_cost, path_cost, parent, child_expand):
        i = parent.position_0 // n
        j = parent.position_0 % n
        if j >= n - 1:
            return None
        right_state = parent.state.copy()
        right_state[i * n + j], right_state[i * n + j + 1] = right_state[i * n + j + 1], right_state[i * n + j]
        temp = Puzzle(right_state, i * n + j + 1, parent_cost, path_cost, "right", parent)
        child_expand.append(temp)

    def child_puzzle(self, n, parent):

        child_expand = []

        if parent.move is None:
            self.move_up(n, self.path_cost, 1, parent, child_expand)
            self.move_down(n, self.path_cost, 1, parent, child_expand)
            self.move_left(n, self.path_cost, 1, parent, child_expand)
            self.move_right(n, self.path_cost, 1, parent, child_expand)
        elif parent.move == "up":
            self.move_up(n, self.path_cost, 1, parent, child_expand)
            self.move_left(n, self.path_cost, 1, parent, child_expand)
            self.move_right(n, self.path_cost, 1, parent, child_expand)
        elif parent.move == "down":
            self.move_left(n, self.path_cost, 1, parent, child_expand)
            self.move_right(n, self.path_cost, 1, parent, child_expand)
            self.move_down(n, self.path_cost, 1, parent, child_expand)
        elif parent.move == "left":
            self.move_left(n, self.path_cost, 1, parent, child_expand)
            self.move_up(n, self.path_cost, 1, parent, child_expand)
            self.move_down(n, self.path_cost, 1, parent, child_expand)
        elif parent.move == "right":
            self.move_right(n, self.path_cost, 1, parent, child_expand)
            self.move_down(n, self.path_cost, 1, parent, child_expand)
            self.move_up(n, self.path_cost, 1, parent, child_expand)

        return child_expand

    def heuristic(self, n):
        state = [int(x) for x in self.state]
        vertical = sum(1 for i in range(len(state)) if state[i] != 0 for j in range(i + 1, len(state)) if
                       state[i] > state[j] and state[j] != 0)
        horizon_string = [state[j] for i in range(n) for j in range(i, len(state), n)]
        horizontal = sum(
            abs(i - goalstate2[int(horizon_string[i])]) for i in range(len(horizon_string)) if horizon_string[i] != 0)
        return vertical // 3 + vertical % 3 + horizontal // 3 + horizontal % 3

**A* SEARCH**

In [26]:
def AstartSearch(initial, goal, n):
    frontier = PriorityQueue(0)
    explored: dict[str, int] = {}
    open_list: dict[str, int] = {}

    frontier.put([initial.heuristic(n) + initial.path_cost, initial])

    while True:
        if frontier.qsize() == 0:
            return explored, "failure", None
        current_puzzle = frontier.get()[-1]
        current_state = ''.join(current_puzzle.state)
        if explored.get(current_state) == 1:
            continue
        if current_puzzle.state == goal:
            path_list = []
            while current_puzzle is not None:
                path_list.append(current_puzzle)
                current_puzzle = current_puzzle.parent
            path_list.reverse()
            return explored, "success", path_list
        explored[current_state] = 1

        for child in current_puzzle.child_puzzle(n, current_puzzle):
            temp = ''.join(child.state)
            if explored.get(temp) is None:
                heuristic = child.heuristic(n)
                if open_list.get(temp) is None or open_list.get(temp) > (child.path_cost + heuristic):
                    frontier.put([child.path_cost + heuristic, child])
                    open_list[temp] = child.path_cost + heuristic

        del current_puzzle
        

**UCS SEARCH**

In [27]:
def UCS(initial, goal, n):
    frontier = PriorityQueue(0)
    explored: dict[str, int] = {}
    open_list: dict[str, int] = {}

    frontier.put([0, initial])

    while True:
        if frontier.qsize() == 0:
            return explored, "failure", None
        current_puzzle = frontier.get()[-1]
        current_state = ''.join(current_puzzle.state)
        if explored.get(current_state) == 1:
            continue
        if current_puzzle.state == goal:
            path_list = []
            while current_puzzle is not None:
                path_list.append(current_puzzle)
                current_puzzle = current_puzzle.parent
            path_list.reverse()
            return explored, "success", path_list
        explored[current_state] = 1

        for child in current_puzzle.child_puzzle(n, current_puzzle):
            temp = ''.join(child.state)
            if explored.get(temp) is None:
                if open_list.get(temp) is None or open_list.get(temp) > child.path_cost:
                    frontier.put([child.path_cost, child])
                    open_list[temp] = child.path_cost

        del current_puzzle
        

**PRINT PATH FUNCTION**

In [28]:
def printPath(c,n):
    for t in c:
        j = 0
        print(t.path_cost)
        for i in t.state:
            if j % n == n-1:

                print(i, end=' ')
                print()
            else:
                print(i, end=' ')
            j += 1
        print("--------------------")

**MAIN FUNCTION**

In [ ]:
while True:
    n = input("Please enter n with n*n is the size of puzzle or 0 to exit: \n")
    if n == '0':
        break
    elif n.isdecimal():
        n = int(n)
        if n == 3 or n == 4 or n == 6:
            npuzzle = input("Please enter the puzzle for example(0,1,2,3,4,5,6,7,8) or only 1 to run default: \n")
            initial = None
            goal = None
            if n == 3:
                goalstate2 = {1: 0, 4: 1, 7: 2, 2: 3, 5: 4, 8: 5, 3: 6, 6: 7, 0: 8}
                if npuzzle == '1':
                    initial = Puzzle(['1', '2', '3', '0', '4', '6', '7', '5', '8'], 3, 0, 0)
                else:
                    puzzle = npuzzle.split(',')
                    position_0 = puzzle.index('0')
                    initial = Puzzle(puzzle, position_0, 0, 0)
                goal = ['1', '2', '3', '4', '5', '6', '7', '8', '0']
            if n == 4:
                goalstate2 = {1: 0, 5: 1, 9: 2, 13: 3, 2: 4, 6: 5, 10: 6, 14: 7, 3: 8, 7: 9, 11: 10, 15: 11, 4: 12,
                              8: 13, 12: 14, 0: 15}
                if npuzzle == '1':
                    initial = Puzzle(
                        ['8', '0', '6', '3', '14', '15', '10', '7', '2', '9', '5', '13', '12', '1', '4', '11'], 1,
                        0, 0)
                else:
                    puzzle = npuzzle.split(',')
                    position_0 = puzzle.index('0')
                    initial = Puzzle(puzzle, position_0, 0, 0)
                goal = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '0']
            if n == 6:
                goalstate2 = {1: 0, 7: 1, 13: 2, 19: 3, 25: 4, 31: 5, 2: 6, 8: 7, 14: 8, 20: 9, 26: 10, 32: 11,
                              3: 12, 9: 13, 15: 14, 21: 15, 27: 16, 33: 17, 4: 18, 10: 19, 16: 20, 22: 21, 28: 22,
                              34: 23, 5: 24, 11: 25, 17: 26, 23: 27, 29: 28, 35: 29, 6: 30, 12: 31, 18: 32, 24: 33,
                              30: 34, 0: 35}
                if npuzzle == '1':
                    initial = Puzzle(
                        ['10', '35', '18', '12', '27', '31', '25', '16', '32', '2', '28', '7', '1', '24', '5', '20',
                         '26', '34', '19', '17', '3', '9', '14', '8', '33', '22', '13', '15', '4', '6', '29', '21',
                         '11', '0', '23', '30'], 33, 0, 0)
                else:
                    puzzle = npuzzle.split(',')
                    position_0 = puzzle.index('0')
                    initial = Puzzle(puzzle, position_0, 0, 0)
                goal = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
                        '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32',
                        '33', '34', '35', '0']
            while True:
                choose = input("Please choose search with UCS: 1 and A*: 2 or 0 to return: \n")
                if choose == '0':
                    break
                elif choose.isdecimal():
                    choose = int(choose)
                    if choose == 1:
                        print("UCS search")
                        time_start = datetime.datetime.now()
                        tracemalloc.start()
                        a, b, c = UCS(initial, goal, n)
                        current, peak = tracemalloc.get_traced_memory()
                        time_end = datetime.datetime.now()
                        print(f'time: {(time_end - time_start).total_seconds() * 1000} milliseconds')
                        print(f"memory now: {current / (1024 * 1024)} MB")
                        print(f"memory max: {peak / (1024 * 1024)} MB")
                        tracemalloc.stop()
                        print(b)
                        printPath(c, n)
                        print("---------------------------")
                    elif choose == 2:
                        print("A* search")
                        time_start = datetime.datetime.now()
                        tracemalloc.start()
                        a, b, c = AstartSearch(initial, goal, n)
                        current, peak = tracemalloc.get_traced_memory()
                        time_end = datetime.datetime.now()
                        print(f'time: {(time_end - time_start).total_seconds() * 1000} milliseconds')
                        print(f"memory now: {current / (1024 * 1024)} MB")
                        print(f"memory max: {peak / (1024 * 1024)} MB")
                        tracemalloc.stop()
                        print(b)
                        if c is not None:
                            printPath(c, n)
                        print("---------------------------")
                    else:
                        print("Please choose correctly")
        else:
            print("n is only 3,4 or 6")

Please enter n with n*n is the size of puzzle or 0 to exit: 
 4
Please enter the puzzle for example(0,1,2,3,4,5,6,7,8) or only 1 to run default: 
 5,7,12,10,2,8,13,6,3,9,4,14,11,0,1,15
Please choose search with UCS: 1 and A*: 2 or 0 to return: 
 2


A* search
time: 747759.517 milliseconds
memory now: 385.0511770248413 MB
memory max: 3366.011067390442 MB
success
0
5 7 12 10 
2 8 13 6 
3 9 4 14 
11 0 1 15 
--------------------
1
5 7 12 10 
2 8 13 6 
3 0 4 14 
11 9 1 15 
--------------------
2
5 7 12 10 
2 8 13 6 
0 3 4 14 
11 9 1 15 
--------------------
3
5 7 12 10 
2 8 13 6 
11 3 4 14 
0 9 1 15 
--------------------
4
5 7 12 10 
2 8 13 6 
11 3 4 14 
9 0 1 15 
--------------------
5
5 7 12 10 
2 8 13 6 
11 3 4 14 
9 1 0 15 
--------------------
6
5 7 12 10 
2 8 13 6 
11 3 4 14 
9 1 15 0 
--------------------
7
5 7 12 10 
2 8 13 6 
11 3 4 0 
9 1 15 14 
--------------------
8
5 7 12 10 
2 8 13 6 
11 3 0 4 
9 1 15 14 
--------------------
9
5 7 12 10 
2 8 0 6 
11 3 13 4 
9 1 15 14 
--------------------
10
5 7 12 10 
2 8 6 0 
11 3 13 4 
9 1 15 14 
--------------------
11
5 7 12 10 
2 8 6 4 
11 3 13 0 
9 1 15 14 
--------------------
12
5 7 12 10 
2 8 6 4 
11 3 13 14 
9 1 15 0 
--------------------
13
5 7 12 10 
2 8 6 4 
11 3 13 14 
9 1